Импортируем необходимые библиотеки, и делаем некоторые предустановки.

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import math
from IPython.display import Javascript
from itertools import combinations
from scipy.stats import ttest_ind

sns.set(style="whitegrid")
pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок

ImportError: No module named matplotlib.pyplot

## Общий обзор
Читаем файл, выводим датасет.

In [2]:
df = pd.read_csv('stud_math.csv')
df.head(10)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
0,GP,F,18,U,NaN,A,4.0,4.0,at_home,teacher,...,-6.0,yes,NaN,no,4.0,3.0,4.0,3.0,6.0,30.0
1,GP,F,17,U,GT3,NaN,1.0,1.0,at_home,other,...,-6.0,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
2,GP,F,15,U,LE3,T,1.0,1.0,at_home,other,...,-6.0,yes,yes,NaN,4.0,3.0,2.0,3.0,10.0,50.0
3,GP,F,15,U,GT3,T,4.0,2.0,health,NaN,...,-9.0,yes,yes,yes,3.0,2.0,2.0,5.0,2.0,75.0
4,GP,F,16,U,GT3,T,3.0,3.0,other,other,...,-6.0,yes,no,no,4.0,3.0,2.0,5.0,4.0,50.0
5,GP,M,16,U,LE3,T,4.0,3.0,services,other,...,-6.0,yes,yes,no,5.0,4.0,2.0,5.0,10.0,75.0
6,GP,M,16,NaN,LE3,T,2.0,2.0,other,other,...,-6.0,yes,yes,no,4.0,4.0,4.0,3.0,0.0,55.0
7,GP,F,17,U,GT3,A,4.0,4.0,other,teacher,...,-6.0,yes,no,no,4.0,1.0,4.0,1.0,6.0,30.0
8,GP,M,15,U,LE3,A,3.0,2.0,services,other,...,-6.0,yes,yes,no,NaN,2.0,2.0,1.0,0.0,95.0
9,GP,M,15,U,NaN,NaN,3.0,4.0,other,other,...,-6.0,yes,yes,no,5.0,5.0,1.0,5.0,0.0,75.0


У нас в распоряжении данные о 395 учениках. С первого взгляда видим, что некоторые поля содержат пустые значения.
И кажется еще кое-что.
Посмотрим информацию о колонках.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 30 columns):
school                 395 non-null object
sex                    395 non-null object
age                    395 non-null int64
address                378 non-null object
famsize                368 non-null object
Pstatus                350 non-null object
Medu                   392 non-null float64
Fedu                   371 non-null float64
Mjob                   376 non-null object
Fjob                   359 non-null object
reason                 378 non-null object
guardian               364 non-null object
traveltime             367 non-null float64
studytime              388 non-null float64
failures               373 non-null float64
schoolsup              386 non-null object
famsup                 356 non-null object
paid                   355 non-null object
activities             381 non-null object
nursery                379 non-null object
studytime, granular    388 non-

Приведем все названия колонок в нижний регистр.

In [4]:
df.columns = map(str.lower, df.columns)

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 30 columns):
school                 395 non-null object
sex                    395 non-null object
age                    395 non-null int64
address                378 non-null object
famsize                368 non-null object
pstatus                350 non-null object
medu                   392 non-null float64
fedu                   371 non-null float64
mjob                   376 non-null object
fjob                   359 non-null object
reason                 378 non-null object
guardian               364 non-null object
traveltime             367 non-null float64
studytime              388 non-null float64
failures               373 non-null float64
schoolsup              386 non-null object
famsup                 356 non-null object
paid                   355 non-null object
activities             381 non-null object
nursery                379 non-null object
studytime, granular    388 non-

Наблюдается два интересных момента.
1. Колонок в нашем датасете 30, в то время как в описании датасета - 29. Видим отсутствующую в описании колонку "studytime, granular", в которой 388 ненулевых значений, так же как и в колонке "studytime". Совпадают ли данные в этих колонках? Выясним позже а пока возьмем на заметку.
2. Не все колонки числового типа отражают абсолютный колчиственный показатель. Например, такие колонки как Medu, Fedu - упрощенное представление об уровне образования.

## Рассмотрим все столбцы

Составим словарь допустимых значений для "номинативных" столбцов. Он понадобится нам для отображения информации на графиках.

"Номинативных" написано в кавычках, потому что, как упомяналось ранее, некторые числовые столбцы не отражают количественный показатель, и такие столбцы в принципе можно считать номинативными

In [6]:
rules = {
    "sex": {
        "title": "пол",
        "values": {
          "F": 'девушки',
          "M": "юноши"   
        }
    },
    "address": {
        "title": "адрес",
        "values": {
          "U": "город",
          "R": "за городом"   
        }
    },
    "famsize": {
        "title": "размер семьи",
        "values": {
          "LE3": "<= 3",
          "GT3": "> 3"
        }
    },
    "pstatus": {
        "title": "родители вместе?",
        "values": {
          "T": "вместе",
          "A": "раздельно"
        }
    },
    "medu": {
        "title": "образование матери",
        "values": {
          "0.0": "0) нет",
          "1.0": "1) 4 кл",
          "2.0": "2) 5-9 кл",
          "3.0": "3) 11 кл / ср.сп.",
          "4.0": "4) высшее",
        }
    },
    "fedu": {
        "title": "образование отца",
        "values": {
          "0.0": "0) нет",
          "1.0": "1) 4 кл",
          "2.0": "2) 5-9 кл",
          "3.0": "3) 11 кл / ср.сп.",
          "4.0": "4) высшее",
        }
    },
    "fjob": {
        "title": "работа отца",
        "values": {
          "teacher": "учитель",
          "health": "здравохранение",
          "services": "гос.служба",
          "at_home": "не работает",
          "other": ".другое",
        }
    },
    "mjob": {
        "title": "работа матери",
        "values": {
          "teacher": "учитель",
          "health": "здравохранение",
          "services": "гос.служба",
          "at_home": "не работает",
          "other": ".другое",
        }
    },
    "reason": {
        "title": "причина выбора школы",
        "values": {
          "home": "близость к дому",
          "reputation": "репутация",
          "course": "об. программа",
          "other": ".другое",
        }
    },
    "guardian": {
        "title": "опекун",
        "values": {
          "mother": "мать",
          "father": "отец",
          "other": "другое",
        }
    },
    "traveltime": {
        "title": "время до школы (мин)",
        "values": {
          "1.0": "1) < 15",
          "2.0": "2) 15-30",
          "3.0": "3) 30-60",
          "4.0": "4) > 60",
        }
    },
    "studytime": {
        "title": "время учебы вне школы (ч)",
        "values": {
          "1.0": "1) < 2",
          "2.0": "2) 2-5",
          "3.0": "3) 5-10",
          "4.0": "4) > 10",
        }
    },
    "studytime, granular": {
        "title": "время учебы вне школы (ч) g",
        "values": {
          "1.0": "1) < 2",
          "2.0": "2) 2-5",
          "3.0": "3) 5-10",
          "4.0": "4) > 10",
        }
    },
    "failures": {
        "title": "внеучебные неудачи",
        "values": {
          "1.0": "1",
          "2.0": "2",
          "3.0": "3",
          "0.0": "другое",}
    },
    "schoolsup": {
        "title": "доп. обр. поддержка",
        "values": {
          "yes": "да",
          "no": "нет"
        }
    },
    "famsup": {
        "title": "семейная обр. поддержка",
        "values": {
          "yes": "да",
          "no": "нет"
        }
    },
    "paid": {
        "title": "доп. платная математика",
        "values": {
          "yes": "да",
          "no": "нет"
        }
    },
    "activities": {
        "title": "доп. внеучебные занятия",
        "values": {
          "yes": "да",
          "no": "нет"
        }
    },
    "nursery": {
        "title": "дет. сад",
        "values": {
          "yes": "да",
          "no": "нет"
        }
    },
    "higher": {
        "title": "вышку хочет?",
        "values": {
          "yes": "да",
          "no": "нет"
        }
    },
    "internet": {
        "title": "интернет есть?",
        "values": {
          "yes": "да",
          "no": "нет"
        }
    },
    "romantic": {
        "title": "отношения",
        "values": {
          "yes": "да",
          "no": "нет"
        }
    },
    "famrel": {
        "title": "семейные отношения",
        "values": {
          "1.0": "1) ужасные",
          "2.0": "2) плохие",
          "3.0": "3) норм",
          "4.0": "4) хорошие",
          "5.0": "5) прекрасные",
        }
    },
    "freetime": {
        "title": "свобода",
        "values": {
          "1.0": "1) оч мало",
          "2.0": "2) мало",
          "3.0": "3) норм",
          "4.0": "4) много",
          "5.0": "5) оч много",
        }
    },
    "goout": {
        "title": "время с друзьями",
        "values": {
          "1.0": "1) оч мало",
          "2.0": "2) мало",
          "3.0": "3) норм",
          "4.0": "4) много",
          "5.0": "5) оч много",
        }
    },
    "health": {
        "title": "здоровье",
        "values": {
          "1.0": "1) ужасное",
          "2.0": "2) плохое",
          "3.0": "3) норм",
          "4.0": "4) хорошее",
          "5.0": "5) прекрасное",
        }
    }
}

Заменим значения в "номинативных" столбцах, чтобы лучше понимать диаграммы.Также заменим пустые значения, чтобы отобразить их на диаграммах. Возьмем -2 вкачестве пустого значение для числовых столбцов.

In [7]:
def change_column_type(name, type):
  if name in list(rules.keys()):
    if type in ['float64', 'int64']:
      df[name] = df[name].astype(str)

def smart_value(name, value):
  if name in list(rules.keys()):
    if value in list(rules[name]['values'].keys()):
      return rules[name]['values'][value]
  return value
  
def empty_value(type, value):
  if type == 'object':
    return '-'
  elif type in ['float64', 'int64']:
    return -2

def change_value(name, type, value):
  if value != value or value == 'nan':
    return empty_value(type, value)
  else:
    return smart_value(name, value)


for column_name in df:
  change_column_type(column_name, df[column_name].dtype)
  df[column_name] = df[column_name].map(
      lambda value: change_value(column_name, df[column_name].dtype, value)
  )

Поскольку в датасете не много данных, и даже в числовых столбцах кол-во уникальных значений не сильно велико, то можно построить для всех столцов столбчатые диаграммы и посмотреть на распределение по признакам.

In [8]:
def explore(df, columns, inrow = 4, percent = True):
  fig, axes = plt.subplots(
    math.ceil(len(columns)/inrow), 
    inrow, 
    figsize=(24, math.ceil(len(columns)/inrow) * 4)
  )

  for column in columns:
    if len(columns) > len(columns) / inrow:
      ax = axes[
        int(math.floor(columns.index(column) + 1) / inrow - 0.00000001),
        int(columns.index(column) % inrow)
      ]
    else:
      ax = axes[int(math.floor(columns.index(column) + 1) / inrow - 0.00000001)]

    df.sort_values(by = [column], inplace = True)
    countplot = sns.countplot(data = df, x = column, ax = ax)
  
    countplot.set(
      ylim=(0, len(df)),
      xlabel = rules[column]['title'] if column in list(rules.keys()) else column, 
      ylabel=''
    )
  
    for p in countplot.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        countplot.annotate(
            f'{int(y)} {f"({int(round(100*y/len(df), 0))}%)" if percent else ""}', 
            (x.mean(), y), 
            ha ='center', 
            va='bottom'
        )
  
  fig.tight_layout() 
  fig.show()

explore(df, ['school', 'sex', 'address', 'famsize', 'pstatus', 
  'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 
  'higher', 'internet', 'romantic'], 6)

explore(df, ['age', 'medu', 'fedu', 'mjob', 'fjob', 'reason', 
  'traveltime', 'studytime', "studytime, granular", 'failures', 'famrel', 'freetime', 
  'goout', 'health'], 2)

explore(df, ['absences', 'score'], 1, percent = False)

SyntaxError: invalid syntax (<ipython-input-8-f83e0a35c0fe>, line 30)

Стобчатые диаграммы наглядно показывают нам распределение учеников по признакам и по количественным показателям. Например мы видим что в нашей выборке больше информации об учениках школы GP, что большинство учеников живут в городе, большинство хотят получить высшее образование, девушек и юношей примерно одинаково. Детально описывать каждый столбец не имеет смысла. Мы видим, что во многих столбцах есть пустые значения. Видим потенциальные выбросы в столбцах "возраст", "образование матери", "образование отца", "семейные отношения". Совсем непонятный столбец "количество внеучебных неудач", мало того непонятно, что он означает, так и не понятно значение 0 - это меньше 1 или больше 3. 

Видим что распределение значений в столбце studytime, granular такое же как и в столбце studytime, но при этом сами значения отличаются. Будем считать что между ними есть однозначное соответствие и не будем рассматривать этот столбец отдельно. И даже удалим его чтобы не мешался.

Есть несколько интересных моментов. 

Значение 40 в столбце "образования отца". Скорее всего это опечатка и там должно быть 4. Заменим.

Значение -1 в столбце "семейные отношения". Скоре всего это опечатка и там должно быть 1. Заменим.

Столбец "пропущенные занятия" выглядит интересно: четных чисел  гораздо больше, чем нечетных, вероятно занятия проходят парами. Один ученик пропустил 385 занятий, вероятно он не посещал занятия вообще никогда.

In [9]:
del df["studytime, granular"]


In [10]:
df['fedu'][df['fedu'] == '40.0'] = rules['fedu']['values']['4.0']

/home/andrei/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df['famrel'][df['famrel'] == '-1.0'] = rules['famrel']['values']['1.0']

/home/andrei/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Выбросы

Давайте найдем выбросы в числовых столбцах пользуясь формулой интерквартильного расстояния.

In [12]:
def show_outliners():
  for column_name in df:
    column = df[column_name]
    if column.dtype in ['int64', 'float64']:
      IQR = column.quantile(0.75) - column.quantile(0.25)
      outliners = {
        "left": column.quantile(0.25) - 1.5 * IQR, 
        "right": column.quantile(0.75) + 1.5 * IQR
      }
      print(column_name, outliners)

    
show_outliners()

('age', {'right': 21.0, 'left': 13.0})
('absences', {'right': 20.0, 'left': -12.0})
('score', {'right': 115.0, 'left': -5.0})


Подобный подход предлагает считать выбросами возраста старше 21. Думаю, сюда же можно включить и 20. Слишком уж мало учеников с таким возрастом, чтобы полагаться на эти значения.

Со значениями выбросов для пропусков пожалуй согласимся.

Выбросов в oценках как видим нет.

Объединим все все возроста от 20 лет в один и будет считать что это 20 лет и страше.

Удалим в столбце пропуски значение более 200 а значение более 20 объединим в одно и будем считать что это 20 и более.

In [13]:
df['age'][df['age'] >= 20] = 20

/home/andrei/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
df["absences"][df["absences"] > 200] = -2

/home/andrei/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
df["absences"][df["absences"] > 20] = 20

/home/andrei/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Корреляция

Посмотрим корреляцию числовых значений на графиках.

In [16]:
sns.pairplot(df, kind = 'reg')

NameError: name 'sns' is not defined

Видим корреляцию с возрастом. Так же наблюдаем небольшую и странную корреляцию с пропусками занятий. Получается, те, кто пропускал больше, сдают экзамен лучше. Почему так? Заметим что среди оценок есть 0. Вероятно это те ученики, кто не пришел на экзамен. Удалим такие записи из нашего датафрейма, поскольку это не правильно учитывать их в анализе результатов экзамена.

In [17]:
df = df[df['score'] > 0]

In [18]:
sns.pairplot(df, kind = 'reg')

NameError: name 'sns' is not defined

Теперь все логично, чем больше пропусков, тем ниже итоговая оценка.

Признаем столбцы возраст и пропуски влияющими на итоговую оценку и оставляем их для дальнейшего построения модели.

#Анализ номинативных переменных

In [19]:
def countplots(df):
  fig, axes = plt.subplots(9, 3, figsize = (22, 60))

  count = 0

  for column_name in df:
    if df[column_name].dtype == 'object':

      ax = axes[count // 3, count % 3]

      df = df.sort_values(by = [column_name])

      sns.boxplot(data = df, x = column_name, y = "score", ax = ax)

      count += 1

  fig.tight_layout() 
  fig.show()


countplots(df)

NameError: global name 'plt' is not defined

Можем заметить, что итоговый резултат лучше у
* юношей
* детей отцов учителей и матерей работающих в сфере здравоохранения
* тех, кто гулял меньше
* тех, у кого здоровье хуже

и так далее, все перечислять не будем, все это видно на диаграммах.

Есть несколько интересных моментов.

Оценка сильно зависит от уровня образования матери, чем уровень выше, тем оценка выше. Тем не менее выделяются результаты учеников, у матерей которых нет образоывания. Интересно почему?

Также интеерсно, почему с образованием отцов такой зависимости нет.

Еще один интересный столбец - количество внеучебных неудач. Чем их меньше, тем результат лучше. Особняком стоит значение 0 (другое), мы не можем знать, что оно означает (меньше 1 или больше 3), следуя общей логике можно предположить, что меньше 1. 



Сделаем тест Стюдента

In [20]:
for col in df.columns:
    
    combs = list(combinations(df[col].unique(), 2))
    
    for a, b in combs:
        a_values = df.loc[df[col] == a, 'score']
        b_values = df.loc[df[col] == b, 'score']        
        pval = ttest_ind(a_values, b_values).pvalue
        if pval <= (0.05 / len(combs)):
            print(f"Найдены статистически значимые различия для колонки {col}")
            break

SyntaxError: invalid syntax (<ipython-input-20-d7cd5aa54562>, line 10)

Оставляем в нашем датасете только статистически значимые столбцы.

In [21]:
res = df.loc[:, ['address', 'age', 'medu', 'fedu', 'mjob', 'fjob', 'studytime', 'failures', 'schoolsup', 'goout', 'absences', 'score']]

In [22]:
res

,address,age,medu,fedu,mjob,fjob,studytime,failures,schoolsup,goout,absences,score
0,город,18,4) высшее,4) высшее,не работает,учитель,2) 2-5,другое,да,4) много,6.0,30.0
1,город,17,1) 4 кл,1) 4 кл,не работает,.другое,2) 2-5,другое,нет,3) норм,4.0,30.0
2,город,15,1) 4 кл,1) 4 кл,не работает,.другое,2) 2-5,3,да,2) мало,10.0,50.0
3,город,15,4) высшее,2) 5-9 кл,здравохранение,-,3) 5-10,другое,нет,2) мало,2.0,75.0
4,город,16,3) 11 кл / ср.сп.,3) 11 кл / ср.сп.,.другое,.другое,2) 2-5,другое,нет,2) мало,4.0,50.0
5,город,16,4) высшее,3) 11 кл / ср.сп.,гос.служба,.другое,2) 2-5,другое,нет,2) мало,10.0,75.0
6,-,16,2) 5-9 кл,2) 5-9 кл,.другое,.другое,2) 2-5,другое,нет,4) много,0.0,55.0
7,город,17,4) высшее,4) высшее,.другое,учитель,2) 2-5,другое,да,4) много,6.0,30.0
8,город,15,3) 11 кл / ср.сп.,2) 5-9 кл,гос.служба,.другое,2) 2-5,другое,нет,2) мало,0.0,95.0
9,город,15,3) 11 кл / ср.сп.,4) высшее,.другое,.другое,2) 2-5,другое,нет,1) оч мало,0.0,75.0
